In [1]:
#importing libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('vader_lexicon')
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Toshiba\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [3]:
#read the data from twitter
df = pd.read_csv('Tweets.csv')
df = df[['tweets','likes','time']]

In [4]:
def text_cleaner(text,num):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub(r"[0-9]", "", newString)
    newString = re.sub(',',' ', newString)
    newString = re.sub('-',' ', newString)
    newString = re.sub('_',' ', newString)
    newString = re.sub('&','and', newString)
    newString = re.sub('[()]', '', newString)
    newString = re.sub(';',' ', newString)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = re.sub(r"Ä¢", "", newString)
    newString = re.sub(r"¬∑", "", newString)
    newString = re.sub(r"\'", "", newString)
    newString = re.sub(r"\"", "", newString)
    newString = re.sub(r"\n", "", newString)
    newString = re.sub(r"\r", "", newString)
    return newString.strip().lower()

In [5]:
X = []
for t in df['tweets']:
    X.append(text_cleaner(t,0))

In [6]:
#Soting list in the column back again
df['tweets'] = pd.DataFrame(X)
df['tweets'].head()

0    farahkhanali plz extend a little help to  figh...
1    nsaina plz extend a little help to  fightssma ...
2    doctorajayita plz extend a little help to  fig...
3    urgent  only  month left   ayaansh is sufferin...
4    priyankajshukla plz extend a little help to  f...
Name: tweets, dtype: object

In [7]:
#Tokenise
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df.tweets))

print(data_words[:4])

[['farahkhanali', 'plz', 'extend', 'little', 'help', 'to', 'fightssma', 'this', 'eidalfitr', 'yr', 'old', 'can', 'even', 'hold', 'his', 'head', 'due', 'to', 'spinal', 'muscular', 'atrophy', 'he', 'is', 'being', 'treated', 'at', 'rainbow', 'hospital', 'hyderabad', 'out', 'of', 'cr', 'cr', 'has', 'been', 'raised', 'for', 'zolgensma', 'amplify', 'https', 'co', 'tbldsdqgk', 'https', 'co', 'eaapneg', 'https', 'co', 'nnkyltyf'], ['nsaina', 'plz', 'extend', 'little', 'help', 'to', 'fightssma', 'yr', 'old', 'can', 'even', 'hold', 'his', 'head', 'due', 'to', 'spinal', 'muscular', 'atrophy', 'he', 'is', 'being', 'treated', 'at', 'rainbow', 'hospital', 'hyderabad', 'out', 'of', 'cr', 'cr', 'has', 'been', 'raised', 'for', 'zolgensma', 'humanityfirst', 'amplify', 'https', 'co', 'tbldsdqgk', 'https', 'co', 'uxmovpuu'], ['doctorajayita', 'plz', 'extend', 'little', 'help', 'to', 'fightssma', 'yr', 'old', 'can', 'even', 'hold', 'his', 'head', 'due', 'to', 'spinal', 'muscular', 'atrophy', 'he', 'is', 'b

In [8]:
import re, nltk, spacy, gensim
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [9]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['farahkhanali extend little help fightssma eidalfitr old can even hold head spinal muscular atrophy be be treat rainbow hospital hyderabad cr cr have be raise zolgensma amplify https co tbldsdqgk https co eaapneg https co nnkyltyf', 'nsaina extend little help fightssma yr old can even hold head spinal muscular atrophy be be treat rainbow hospital hyderabad cr cr have be raise zolgensma humanityfirst amplify https co tbldsdqgk https co uxmovpuu']


In [10]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import text
from pprint import pprint

In [11]:
add_stop_words = ['https', 'get', 'many', 'have', 'hold', 'sir','need','help','plz','make',
                  'available','pls','spinal','muscular','atrophy','vjyxqgt','fightssma','sma','asr']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
vectorizer = CountVectorizer(analyzer='word',
                             token_pattern='[a-zA-Z0-9]{3,}',
                             stop_words=stop_words# num chars > 3
                            )

In [12]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [13]:
word_list = vectorizer.get_feature_names()
count_list = data_vectorized.toarray().sum(axis=0)

In [14]:
print(dict(zip(word_list,count_list)))

{'aabha': 1, 'aamer': 1, 'aamiredresy': 1, 'aayanshfightsma': 3, 'able': 10, 'access': 3, 'accord': 1, 'account': 1, 'actforsma': 2, 'action': 1, 'activity': 2, 'actor': 1, 'actormadhavan': 1, 'actress': 2, 'adequate': 1, 'adeseal': 1, 'adnansamilive': 1, 'adult': 5, 'advance': 1, 'advancement': 1, 'aebsipu': 4, 'aeuduhovo': 1, 'affect': 4, 'afford': 52, 'age': 10, 'agfwzwz': 1, 'ago': 1, 'agrimonious': 1, 'ahead': 1, 'ahin': 1, 'ajmc': 1, 'ajowdcva': 1, 'akin': 1, 'akinakinozu': 1, 'akinakinozufan': 1, 'akzhgavz': 1, 'alev': 4, 'alifazal': 1, 'allow': 1, 'alluarjun': 4, 'alok': 2, 'alperen': 1, 'alpereneumutol': 1, 'amazing': 7, 'ambassadortai': 1, 'ambulatory': 2, 'amd': 1, 'american': 1, 'amifpumie': 2, 'amniotic': 1, 'amplify': 36, 'amritarao': 1, 'anahita': 1, 'andamp': 91, 'anhmdhkuh': 1, 'anjtsb': 1, 'ankitbatramusic': 1, 'ankurfactorial': 1, 'antisense': 1, 'anuragkashyap': 1, 'anuragspparty': 1, 'appeal': 4, 'application': 1, 'apply': 1, 'approach': 1, 'approve': 1, 'approxima

In [15]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
# Build LDA Model
fixed_seed = 100
lda_model = LatentDirichletAllocation(n_components=5,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',
                                      random_state=np.random.RandomState(fixed_seed),          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=5, n_jobs=-1,
                          random_state=RandomState(MT19937) at 0x1AED7D4F258)


In [16]:
# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df.tweets))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head().style.applymap(color_green).applymap(make_bold)
df_document_topics

In [17]:
df_document_topic=df_document_topic.loc[:,['dominant_topic']]
df_document_topic1= df_document_topic.reset_index()

In [18]:
#df_document_topic1.head(10)

In [19]:
df_topic_distribution = df_document_topic1['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

Topic Num  Num Documents
0          4            177
1          0            124
2          3             90
3          2             36
4          1             26

In [20]:
# # Visulaiise the topic keywords
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  38.960688
0       51.163364 -159.968018       2        1  26.388282
3      -73.144554   21.523815       3        1  22.835281
2      -66.410133 -110.606438       4        1   6.077006
1       54.417324   12.443444       5        1   5.738744, topic_info=           Term       Freq      Total Category  logprob  loglift
53       andamp  81.000000  81.000000  Default  30.0000  30.0000
213        cost  77.000000  77.000000  Default  29.0000  29.0000
1309    therapy  61.000000  61.000000  Default  28.0000  28.0000
715      little  99.000000  99.000000  Default  27.0000  27.0000
940     patient  60.000000  60.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
287   diagnosis   1.646625   3.478359   Topic5  -5.5350   2.1101
979   potential   1.567061   3.161862   Topic5  -5.5845   2.1560
708        life   3.983883  40.244377   Topic5  -4.6515   0.5452
420       fight   2.979233  24.857745   Topic5  -4.9421   0.7364
173       child   1.989813  31.216108   Topic5  -5.3457   0.1050

[259 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
3         3  0.654225  aayanshfightsma
4         2  0.138049             able
4         4  0.828292             able
5         5  0.663177           access
17        5  0.717663            adult
...     ...       ...              ...
1491      5  0.512505       znrpwlgxma
1495      1  0.579775        zolgensma
1495      2  0.062341        zolgensma
1495      3  0.330409        zolgensma
1495      4  0.018702        zolgensma

[314 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 3, 2])

In [21]:
pyLDAvis.save_html(panel, 'lda.html')

In [22]:
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

Word 0     Word 1  Word 2     Word 3     Word 4   Word 5     Word 6  \
Topic 0     old      month   raise     suffer     battle     life       year   
Topic 1    type        set    life   training      model    adult        use   
Topic 2   share       able   fight       save     dmytro  disease  zolgensma   
Topic 3  andamp  treatment    cost    therapy    patient   spread     muscle   
Topic 4   raise        old  little  zolgensma  tbldsdqgk    treat       head   

            Word 7     Word 8   Word 9        Word 10  Word 11   Word 12  \
Topic 0       save        day    child          crore     drug   wrpjtkw   
Topic 1   research     severe     gune       movement    learn     fight   
Topic 2    genetic   medicine  ayaansh  neuromuscular    child     donor   
Topic 3       gene  zolgensma   dollar            tag   weaken   comment   
Topic 4  hyderabad   hospital  rainbow       medicine  support  donation   

          Word 13     Word 14  Word 15  Word 16    Word 17  Word 18 Word 19  
Topic 0      baby   treatment   donate     come       type  support   share  
Topic 1      live  nusinersen     form  drxcdsn    trouble  feeding   limit  
Topic 2  birthday       reach   battle     want       doze     haly   story  
Topic 3   unaware      nefeye  country    sesol  selective  disease   world  
Topic 4      fund      afford    small      die        kid    covid    kill